In [2]:
import pandas as pd
import glob
import openpyxl
import re

# Misc Functions

## Replace Function

In [74]:
def replace_func(x):
    if isinstance(x, float):
        return x
    else:
        if x.strip() == "NA" or x is None:
            return 0
        else:
            return x

## Function to check if a row is a table note or footnote

In [ ]:
def is_note_or_footnote(name):
    name_str = str(name)
    table_note_pattern = r'^\d+:'
    footnote_pattern = r'^[a-z]\)|#\)|\*\)'

    if re.match(table_note_pattern, name_str):
        return "Table notes"
    elif re.match(footnote_pattern, name_str):
        return "Footnotes"
    elif name_str.strip().lower() in ["footnotes", "notes"]:
        return name_str.strip().capitalize()
    return None

## Function to extract the 'Notes' section and return it as a DataFrame1

In [ ]:
def extract_notes(file_path):
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    notes_data = []
    notes_started = False

    for row in sheet.iter_rows(values_only=True):
        if notes_started:
            notes_data.append(row)
        elif row[0] and str(row[0]).strip().lower() == "notes":
            notes_started = True
            notes_data.append(row)

    notes_df = pd.DataFrame(notes_data)

    return notes_df

# Function to unmerge cells and create a dictionary from footnotes

In [ ]:
def extract_footnotes(file_path):
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    footnotes_dict = {}
    
    for row in range(2, sheet.max_row + 1):
        footnote_key = sheet.cell(row=row, column=1).value
        if footnote_key:
            footnote_value = sheet.cell(row=row, column=2).value
            if footnote_value:
                footnotes_dict[footnote_key.strip()] = footnote_value.strip()
            for merged_cells in list(sheet.merged_cells.ranges):
                if merged_cells.min_row == row and merged_cells.min_col == 2:
                    sheet.unmerge_cells(str(merged_cells))
    
    workbook.save(file_path)
    return footnotes_dict

## Function to process and read Excel files

In [ ]:
def read_and_process_excel(file_path):
    footnotes_dict = extract_footnotes(file_path)
    df_full = pd.read_excel(file_path, header=None)
    df = pd.read_excel(file_path, header=1)
    df.columns = df.columns.map(str)
    df.set_index(df.columns[0], inplace=True)

    def replace_reference(reference):
        if pd.isna(reference):
            return reference
        matches = re.findall(r'([a-zA-Z]\)|\*\)|#\))', reference)
        replaced_reference = ", ".join([footnotes_dict.get(match.strip(')'), match) for match in matches])
        return replaced_reference.strip()

    if 'Reference' in df.columns:
        df['Reference'] = df['Reference'].apply(replace_reference)

    # Remove rows that are identified as "Table notes", "Footnotes", or "Notes"
    df = df[~df.index.to_series().apply(is_note_or_footnote).notnull()]

    # Ensure no "Table notes" remains in the Country column
    df = df[~df.index.str.lower().str.contains('table notes')]
    

    country = []
    year = []
    quantity = []
    reference = []
    unit = []

    unit_value = df_full.iloc[0, 1].strip().title()

    for col in df.columns:
        if re.match(r'^\d{4}$', col):
            country.extend(df.index)
            year.extend([col] * len(df))
            quantity_col = col + ".1"
            quantity.extend(df[quantity_col] if quantity_col in df.columns else ["NA"] * len(df))
            reference.extend(df[col])
            unit.extend([unit_value] * len(df))

    processed_df = pd.DataFrame({
        'Country': country,
        'Year': year,
        'Quantity': quantity,
        'Unit': unit,
        'Reference': reference
    })

    processed_df.reset_index(drop=True, inplace=True)
    processed_df['Quantity'] = processed_df['Quantity'].apply(replace_func).fillna(0).astype(float)
    processed_df['Year'] = processed_df['Year'].astype(int)
    
    return processed_df

## Function to organize the notes and footnotes into the required format

In [ ]:
def organize_notes(notes_df):
    table_notes_df = notes_df[notes_df.index.to_series().apply(is_note_or_footnote) == "Table notes"].drop_duplicates()
    footnotes_df = notes_df[notes_df.index.to_series().apply(is_note_or_footnote) == "Footnotes"].drop_duplicates()
    additional_notes_df = notes_df[notes_df.index.to_series().apply(is_note_or_footnote) == "Notes"].drop_duplicates()

    footnotes_df = footnotes_df.sort_index()

    organized_notes = pd.DataFrame()
    if not additional_notes_df.empty:
        notes_section = pd.DataFrame({"Table Notes": ["Notes"]})
        organized_notes = pd.concat([organized_notes, notes_section])
        additional_notes_df.insert(0, "Table Notes", additional_notes_df.iloc[:, 0])
        organized_notes = pd.concat([organized_notes, additional_notes_df.iloc[:, :1]])

    if not table_notes_df.empty:
        table_notes_section = pd.DataFrame({"Table Notes": ["Table notes"]})
        organized_notes = pd.concat([organized_notes, table_notes_section])
        table_notes_df.insert(0, "Table Notes", table_notes_df.iloc[:, 0])
        organized_notes = pd.concat([organized_notes, table_notes_df.iloc[:, :1]])

    if not footnotes_df.empty():
        footnotes_section = pd.DataFrame({"Table Notes": ["Footnotes"]})
        organized_notes = pd.concat([organized_notes, footnotes_section])
        footnotes_df.insert(0, "Table Notes", footnotes_df.iloc[:, 0])
        organized_notes = pd.concat([organized_notes, footnotes_df.iloc[:, :1]])

    return organized_notes

## Main function to process files, extract notes, and merge everything into the final output

In [ ]:
def process_files_and_merge_notes():
    excel_files = glob.glob('files/*.xlsx')
    merged_df = pd.DataFrame()
    combined_notes = []

    for file in excel_files:
        df = read_and_process_excel(file)
        merged_df = pd.concat([merged_df, df])

        notes_df = extract_notes(file)
        
        for _, row in notes_df.iterrows():
            note_str = " ".join([str(x) for x in row if x is not None])
            combined_notes.append(note_str)

    combined_notes = list(set(combined_notes))

    filtered_notes = [note for note in combined_notes if re.match(r'^\d+:', note)]
    filtered_notes.sort(key=lambda x: int(re.match(r'(\d+):', x).group(1)))

    note_rows = pd.DataFrame({"Table Notes": filtered_notes})
    note_rows["Country"] = ""
    note_rows["Year"] = ""
    note_rows["Quantity"] = ""
    note_rows["Unit"] = ""
    note_rows["Reference"] = ""

    merged_df = pd.concat([merged_df, note_rows], ignore_index=True)

    merged_df = merged_df.sort_values(by=['Country', 'Year'], na_position='last')

    extracted_star_note = next((note.split('*)')[1].strip() for note in combined_notes if note.startswith('*)')), None)
    extracted_hash_note = next((note.split('#)')[1].strip() for note in combined_notes if note.startswith('#)')), None)
    letter_notes = {letter: next((note.split(f'{letter})')[1].strip() for note in combined_notes if note.startswith(f'{letter})')), None) for letter in 'abcdefghijklmnopqrstuvwxyz'}

    def replace_references(row):
        if isinstance(row['Reference'], str):
            replacements = []

            if '*' in row['Reference'] and extracted_star_note:
                replacements.append(extracted_star_note)

            if '#' in row['Reference'] and extracted_hash_note:
                replacements.append(extracted_hash_note)

            for letter in 'abcdefghijklmnopqrstuvwxyz':
                if f'({letter})' in row['Reference'] and letter_notes[letter]:
                    replacements.append(letter_notes[letter])

            if replacements:
                row['Reference'] = ', '.join(replacements)

        return row

    merged_df = merged_df.apply(replace_references, axis=1)
    output_file = 'output/final.xlsx'
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        merged_df.to_excel(writer, sheet_name='Data', index=False)

    final_df = pd.read_excel(output_file, sheet_name='Data')
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    print(final_df.head())
    print(f'Finalfile saved to {output_file}')




# Run the process

In [ ]:
process_files_and_merge_notes()

## Testing

In [136]:
def CreateNotes(Notes_df):
    combined_notes = []
    for _, row in Notes_df.iterrows():
        note_str = " ".join([str(x) for x in row if x is not None])
        combined_notes.append(note_str)
    combined_notes = [re.sub(r'\s+', ' ', note).strip() for note in combined_notes]
    Notes_dict = {}
    key = ""
    for note in combined_notes:
        if isinstance(note, str):
            if note in ["Notes", "Table notes", "Footnotes"]:
                Notes_dict[note] = []
                key =  note
            elif key != "":
                Notes_dict[key].append(note)
    return Notes_dict

In [137]:
def GetKeys(df):
    units = ["tonnes", "kilograms", "Carats", "cubic metres"]
    FirstRow = df.iloc[0].astype(str).tolist()
    # Title extraction
    for i in FirstRow:
        if i.startswith("Production of "):
            idx = len("Production of ")
            Title = i[idx:].strip().title()
        if any(unit in i for unit in units):
            Unit = i.strip().title()
            if FirstRow.index(i) > 1:
                subtype = FirstRow[1]
            else:
                subtype = None
    return Title, Unit, subtype

In [138]:
def replace_references(Ref):
    ReplaceRef = []
    
    for i in Ref:
        if isinstance(i, str):
            # Extract references manually
            ListofRef = re.findall(r'\((.*?)\)', i)
            
            # Manually add * and # if they exist in the string
            if "*" in i:
                ListofRef.append("*")
            if "#" in i:
                ListofRef.append("#")
            
            # Collect the corresponding footnotes
            temp = [extractedfootnotes[j][0] for j in ListofRef if j in extractedfootnotes]
            
            # Append the joined footnotes or an empty string if no match
            ReplaceRef.append("; ".join(temp) if temp else "")
        else:
            ReplaceRef.append("")
    
    return ReplaceRef


In [165]:
def replace_nan_with_previous(index_list):
    previous_value = None
    for i, value in enumerate(index_list):
        if value == 'nan':
            # Replace 'nan' with the previous value and add '.0'
            if previous_value is not None:
                index_list[i] = f"{previous_value}.0"
        else:
            # Update previous_value to the current non-'nan' value
            previous_value = value
    
    return index_list

In [191]:
def processdf(df, Notes_dict, title, sub):
    if sub != None:
        Metal = title + " "+ sub
    else:
        Metal = title
    ListYear, Ref, Qty = [], [], []
    CompleteListYear = []
    
    Finaldf = filter_df.copy()
    Finaldf.columns = Finaldf.iloc[1].astype(str)
    Finaldf = Finaldf.iloc[2:].reset_index(drop=True)
    Finaldf.columns = replace_nan_with_previous(Finaldf.columns.tolist())
    for i in Finaldf.columns:
        if re.match(r'^\d{4}$', i):
           ListYear.append(i)
    Countries = Finaldf[Finaldf.columns[0]].tolist()*len(ListYear)
    for j in ListYear:
        Ref.extend(Finaldf[j].tolist())

    QtyList = [i+".0" for i in ListYear]
    for k in QtyList:
        Qty.extend(Finaldf[k].tolist())
        
        CompleteListYear.extend([ListYear[QtyList.index(k)]]* len(Finaldf[k].tolist()))
    
    MetalList = [Metal]*len(Qty)

    NewRef = replace_references(Ref)
    
    return Countries, Qty, NewRef, CompleteListYear, MetalList

In [196]:
excel_files = glob.glob('files/*.xlsx')
Countries, QtyList, NewRef, ListYear, Metal_Notes, Name = [], [], [], [], [], []
Aggregate = {}
PreviousTitle = None
Continue = False
for index , file in enumerate(excel_files):
    df = pd.read_excel(file, header=None)
    Title, Unit, subtype = GetKeys(df)
    idx = df[df.iloc[:,0] == "Notes"].index[0]
    filter_df = df.loc[:idx-1]
    Notes_df = df.loc[idx:].fillna("")
    extractedfootnotes = {key: [note.split(')')[1].strip() for note in Notes_dict["Footnotes"] if note.startswith(key)] for key in set(note.split(')')[0].strip() for note in Notes_dict["Footnotes"])}
    Notes_dict = CreateNotes(Notes_df)
    _Countries, _QtyList, _NewRef, _ListYear, _Metal = processdf(df, Notes_dict, Title, subtype)
    Metal_Notes.extend(Notes_dict["Table notes"])
    Countries.extend(_Countries)
    QtyList.extend(_QtyList)
    NewRef.extend(_NewRef)
    ListYear.extend(_ListYear)
    Name.extend(_Metal)
Metal_Notes = list(dict.fromkeys(Metal_Notes))
AdditionalLen = [" "] * (len(QtyList) - len(Metal_Notes))
UnitList = [Unit] * len(QtyList)
Metal_Notes.extend(AdditionalLen)
Resultdf = pd.DataFrame(
{
    "Mineral/Metal": Name,
    "Country": Countries,
    "Year" : ListYear,
    "Quantity": QtyList,
    "Units" : UnitList,
    "References": NewRef,
})
Resultdf["Year"] = Resultdf["Year"].apply(replace_func).fillna(0).astype(int)
Resultdf["Quantity"] = Resultdf["Quantity"].apply(replace_func).fillna(0).astype(float)
Resultdf = Resultdf.sort_values(by=["Mineral/Metal", "Country", "Year"])
Resultdf["Table Notes"] = Metal_Notes
Resultdf.to_excel("output/final.xlsx", sheet_name = "Metals in the World", index=False)

C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains

UnboundLocalError: cannot access local variable 'Unit' where it is not associated with a value

In [183]:
Aggregate

{'Iron Ore':        Country  Year  Quantity            Units  \
 1287   Albania  1979  530000.0  Tonnes (Metric)   
 1342   Albania  1980  550000.0  Tonnes (Metric)   
 1397   Albania  1981  600000.0  Tonnes (Metric)   
 1452   Albania  1982  702000.0  Tonnes (Metric)   
 1507   Albania  1983  850000.0  Tonnes (Metric)   
 ...        ...   ...       ...              ...   
 405   Zimbabwe  2007   79109.0  Tonnes (Metric)   
 463   Zimbabwe  2008    2919.0  Tonnes (Metric)   
 521   Zimbabwe  2009       0.0  Tonnes (Metric)   
 579   Zimbabwe  2010      28.0  Tonnes (Metric)   
 637   Zimbabwe  2011       0.0  Tonnes (Metric)   
 
                             References  \
 1287  Nickeliferous iron ore; Estimate   
 1342  Nickeliferous iron ore; Estimate   
 1397            Nickeliferous iron ore   
 1452            Nickeliferous iron ore   
 1507            Nickeliferous iron ore   
 ...                                ...   
 405                                      
 463              